# Air Quality Index and Viral Load comparison in Indian Background:

According to the article in https://scroll.in/article/978454/coronavirus-the-record-high-number-of-new-cases-in-delhi-is-a-direct-effect-of-air-pollution and theGuardian article (https://www.theguardian.com/world/2020/nov/11/delhi-covid-crisis-worsened-by-soaring-pollution-levels) , Delhi's current conditions in the pandemic has a direct relation with the air pollution as the the Covid-19 virus can directly "piggyback" on the PM 2.5 Particles and more severely.
Here, we will have a look at the Air Quality Index in the past 5 years of different cities of India and conclude on the expected Air pollution in the city of Delhi, in particular.
Further since due to Lockdown, the air pollution was comparatively low in all the cities, we will see the Covid cases in Delhi in particular and if it has any correlation with air quality index. 
Further, based on the pollution trends in different cities(increasing or decreasing ), we can find useful insights about the third wave, expected this year around October. 


In [34]:
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

#Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

#For Missing Value and Feature Engineering
from sklearn.feature_selection import SelectKBest, chi2, f_classif, VarianceThreshold
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

import time
%matplotlib inline

### Datasets:

In [51]:
#air quality index 
aqi=pd.read_csv("../input/air-quality-data-in-india/city_hour.csv")

In [38]:
aqi.head()

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01 01:00:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,122.07,NaN,0.0,0.0,0.0,NaN,NaN
1,Ahmedabad,2015-01-01 02:00:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,85.90,NaN,0.0,0.0,0.0,NaN,NaN
2,Ahmedabad,2015-01-01 03:00:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,52.83,NaN,0.0,0.0,0.0,NaN,NaN
3,Ahmedabad,2015-01-01 04:00:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,39.53,153.58,0.0,0.0,0.0,NaN,NaN
4,Ahmedabad,2015-01-01 05:00:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,32.63,NaN,0.0,0.0,0.0,NaN,NaN


In [52]:

# air quality index from 2015-2020 in Delhi.
aqi=aqi[aqi.City =='Delhi']

In [11]:
pd.DatetimeIndex(aqi['Datetime']).year

Int64Index([2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
            ...
            2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],
           dtype='int64', name='Datetime', length=707875)

In [42]:
#covid vaccinations so far
covidstate=pd.read_csv("../input/covid19-in-india/covid_vaccine_statewise.csv")

In [16]:
covidstate.head()

,Updated On,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,Total Sputnik V Administered,AEFI,18-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,NaN,NaN,NaN,NaN,NaN,48276.0
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,NaN,NaN,NaN,NaN,NaN,58604.0
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,NaN,NaN,NaN,NaN,NaN,99449.0
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,NaN,NaN,NaN,NaN,NaN,195525.0
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,NaN,NaN,NaN,NaN,NaN,251280.0


In [43]:
#covid cases in districts
coviddist=pd.read_csv("../input/covid19-corona-virus-india-dataset/district_level_latest.csv")

In [20]:
coviddist.head()

,SlNo,State_Code,State,District_Key,District,Confirmed,Active,Recovered,Deceased,Migrated_Other,Delta_Confirmed,Delta_Active,Delta_Recovered,Delta_Deceased,District_Notes,Last_Updated
0,0,UN,State Unassigned,UN_Unassigned,Unassigned,0,0,0,0,0,0,0,0,0,NaN,NaN
1,1,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobars,0,0,0,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
2,2,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,1,0,1,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
3,3,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,51,19,32,0,0,0,0,0,0,District-wise numbers are out-dated as cumulat...,NaN
4,0,AP,Andhra Pradesh,AP_Foreign Evacuees,Foreign Evacuees,434,0,434,0,0,0,0,0,0,NaN,NaN


In [44]:
#covid cases in states daily
coviddaily=pd.read_csv("../input/covid19-corona-virus-india-dataset/state_level_daily.csv")

In [22]:
coviddaily.head()

,Unnamed: 0,Date,State,Confirmed,Deceased,Recovered,State_Name
0,0,01-Apr-20,AN,0,0,0,Andaman and Nicobar Islands
1,1,01-Apr-20,AP,67,0,1,Andhra Pradesh
2,2,01-Apr-20,AR,0,0,0,Arunachal Pradesh
3,3,01-Apr-20,AS,15,0,0,Assam
4,4,01-Apr-20,BR,3,0,0,Bihar


In [45]:
#testing in states daily 
covidtests=pd.read_csv("../input/covid19-corona-virus-india-dataset/tests_state_wise.csv")

In [24]:
covidtests.head()

,Updated On,State,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Cumulative People In Quarantine,Total People Currently in Quarantine,Tag (People in Quarantine),...,Corona Enquiry Calls,Num Calls State Helpline,Source1,Source2,Source3,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
0,17/04/2020,Andaman and Nicobar Islands,1403.0,Samples Sent,12.0,1210,181.0,NaN,NaN,NaN,...,NaN,NaN,https://t.me/indiacovid/2550,NaN,NaN,0.86%,3.53,3534.0,117,397000.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,Samples Sent,27.0,NaN,246.0,NaN,614.0,Institutional,...,NaN,280.0,https://t.me/indiacovid/3147?single,NaN,NaN,1.01%,6.75,6748.0,99,397000.0
2,27/04/2020,Andaman and Nicobar Islands,2848.0,Samples Sent,33.0,NaN,106.0,NaN,724.0,Institutional,...,NaN,298.0,https://t.me/indiacovid/3365?single,NaN,NaN,1.16%,7.17,7174.0,86,397000.0
3,01/05/2020,Andaman and Nicobar Islands,3754.0,Samples Sent,33.0,NaN,199.0,NaN,643.0,Institutional,...,NaN,340.0,https://t.me/indiacovid/3781,NaN,NaN,0.88%,9.46,9456.0,114,397000.0
4,16/05/2020,Andaman and Nicobar Islands,6677.0,Samples Sent,33.0,NaN,136.0,NaN,16.0,Institutional,...,NaN,471.0,https://t.me/indiacovid/4925,NaN,NaN,0.49%,16.82,16819.0,202,397000.0


### Data Preprocessing:

In [25]:
#helper function to view the table content details
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
def tbl_report(tbl, cols=None, card=52):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='NA'
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return pretty_print(df[cols])
    return pretty_print(df)

In [26]:
tbl_report(aqi15)

Table Shape (67174, 16)


,dtype,nulls,uniques,num_uniques,value_counts
City,object,0,"[Ahmedabad, Bengaluru, Chennai, Delhi, Gurugram, Hyderabad, Lucknow, Mumbai, Patna]",9,Bengaluru:8759Chennai:8759Delhi:8759Ahmedabad:8759Mumbai:8759Lucknow:8759Hyderabad:8674Patna:5121Gurugram:825
Datetime,object,0,"[2015-01-01 01:00:00, 2015-01-01 02:00:00, 2015-01-01 03:00:00, 2015-01-01 04:00:00, 2015-01-01 05:00:00, 2015-01-01 06:00:00, 2015-01-01 07:00:00, 2015-01-01 08:00:00, 2015-01-01 09:00:00, 2015-01-01 10:00:00, 2015-01-01 11:00:00, 2015-01-01 12:00:00, 2015-01-01 13:00:00, 2015-01-01 14:00:00, 2015-01-01 15:00:00, 2015-01-01 16:00:00, 2015-01-01 17:00:00, 2015-01-01 18:00:00, 2015-01-01 19:00:00, 2015-01-01 20:00:00, 2015-01-01 21:00:00, 2015-01-01 22:00:00, 2015-01-01 23:00:00, 2015-01-02 00:00:00, 2015-01-02 01:00:00, 2015-01-02 02:00:00, 2015-01-02 03:00:00, 2015-01-02 04:00:00, 2015-01-02 05:00:00, 2015-01-02 06:00:00, 2015-01-02 07:00:00, 2015-01-02 08:00:00, 2015-01-02 09:00:00, 2015-01-02 10:00:00, 2015-01-02 11:00:00, 2015-01-02 12:00:00, 2015-01-02 13:00:00, 2015-01-02 14:00:00, 2015-01-02 15:00:00, 2015-01-02 16:00:00, 2015-01-02 17:00:00, 2015-01-02 18:00:00, 2015-01-02 19:00:00, 2015-01-02 20:00:00, 2015-01-02 21:00:00, 2015-01-02 22:00:00, 2015-01-02 23:00:00, 2015-01-03 00:00:00, 2015-01-03 01:00:00, 2015-01-03 02:00:00, 2015-01-03 03:00:00, 2015-01-03 04:00:00, 2015-01-03 05:00:00, 2015-01-03 06:00:00, 2015-01-03 07:00:00, 2015-01-03 08:00:00, 2015-01-03 09:00:00, 2015-01-03 10:00:00, 2015-01-03 11:00:00, 2015-01-03 12:00:00, 2015-01-03 13:00:00, 2015-01-03 14:00:00, 2015-01-03 15:00:00, 2015-01-03 16:00:00, 2015-01-03 17:00:00, 2015-01-03 18:00:00, 2015-01-03 19:00:00, 2015-01-03 20:00:00, 2015-01-03 21:00:00, 2015-01-03 22:00:00, 2015-01-03 23:00:00, 2015-01-04 00:00:00, 2015-01-04 01:00:00, 2015-01-04 02:00:00, 2015-01-04 03:00:00, 2015-01-04 04:00:00, 2015-01-04 05:00:00, 2015-01-04 06:00:00, 2015-01-04 07:00:00, 2015-01-04 08:00:00, 2015-01-04 09:00:00, 2015-01-04 10:00:00, 2015-01-04 11:00:00, 2015-01-04 12:00:00, 2015-01-04 13:00:00, 2015-01-04 14:00:00, 2015-01-04 15:00:00, 2015-01-04 16:00:00, 2015-01-04 17:00:00, 2015-01-04 18:00:00, 2015-01-04 19:00:00, 2015-01-04 20:00:00, 2015-01-04 21:00:00, 2015-01-04 22:00:00, 2015-01-04 23:00:00, 2015-01-05 00:00:00, 2015-01-05 01:00:00, 2015-01-05 02:00:00, 2015-01-05 03:00:00, 2015-01-05 04:00:00, ...]",8759,NA
PM2.5,float64,24087,"[nan, 36.6, 40.63, 57.07, 81.3, 105.53, 118.33, 119.4, 137.43, 120.0, 103.8, 72.8, 51.1, 68.73, 76.03, 74.9, 80.47, 87.8, 90.53, 83.53, 68.4, 67.57, 61.13, 63.43, 54.37, 49.2, 67.07, 90.93, 100.37, 112.63, 93.57, 100.0, 89.1, 66.23, 78.93, 87.77, 77.83, 66.47, 65.73, 78.57, 72.37, 79.97, 88.07, 73.43, 58.73, 51.27, 49.6, 49.3, 40.77, 37.27, 55.77, 73.83, 113.77, 167.67, 193.77, 160.83, 141.37, 128.77, 109.83, 79.33, 61.9, 64.8, 63.5, 73.93, 89.17, 101.8, 102.33, 101.67, 96.57, 72.67, 53.63, 48.13, 40.57, 45.87, 55.8, 76.6, 172.7, 189.2, 222.07, 246.43, 234.37, 212.57, 169.33, 123.47, 93.77, 103.63, 104.27, 102.53, 117.57, 118.0, 103.57, 83.9, 64.2, 61.53, 62.07, 68.1, 53.87, 70.37, 173.3, 275.87, ...]",15694,NA
PM10,float64,55330,"[nan, 70.46, 12.76, 58.31, 59.85, 72.5, 76.88, 61.43, 43.46, 53.4, 75.16, 161.07, 179.76, 196.08, 236.8, 238.76, 246.88, 246.32, 257.7, 238.48, 214.11, 176.03, 144.72, 116.12, 107.78, 114.41, 149.14, 170.3, 200.2, 194.2, 195.6, 189.14, 185.09, 167.58, 168.68, 171.12, 175.18, 87.56, 103.81, 120.48, 131.38, 133.6, 109.41, 105.77, 77.88, 85.69, 67.72, 60.6, 68.34, 54.73, 59.2, 65.66, 69.01, 69.26, 85.38, 70.05, 58.37, 65.62, 61.51, 55.1, 42.93, 39.79, 45.7, 41.27, 34.05, 28.58, 26.46, 21.64, 13.82, 9.2, 14.38, 22.35, 21.76, 25.79, 27.27, 17.82, 15.03, 10.0, 6.84, 24.98, 41.26, 28.92, 19.6, 10.97, 8.2, 4.91, 15.81, 14.77, 9.04, 8.75, 9.32, 7.3, 4.81, 4.41, 6.53, 5.85, 4.87, 4.32, 4.51, 6.25, ...]",8793,NA
NO,float64,12614,"[1.0, 0.02, 0.08, 0.3, 0.12, 0.33, 0.45, 1.03, 1.47, 2.05, 2.27, 1.73, 1.72, 1.85, 0.95, 0.87, 0.8, 0.47, 0.53, 0.83,

In [27]:
tbl_report(covidstate)

Table Shape (5144, 18)


,dtype,nulls,uniques,num_uniques,value_counts
Updated On,object,0,"[16/01/2021, 17/01/2021, 18/01/2021, 19/01/2021, 20/01/2021, 21/01/2021, 22/01/2021, 23/01/2021, 24/01/2021, 25/01/2021, 26/01/2021, 27/01/2021, 28/01/2021, 29/01/2021, 30/01/2021, 31/01/2021, 01/02/2021, 02/02/2021, 03/02/2021, 04/02/2021, 05/02/2021, 06/02/2021, 07/02/2021, 08/02/2021, 09/02/2021, 10/02/2021, 11/02/2021, 12/02/2021, 13/02/2021, 14/02/2021, 15/02/2021, 16/02/2021, 17/02/2021, 18/02/2021, 19/02/2021, 20/02/2021, 21/02/2021, 22/02/2021, 23/02/2021, 24/02/2021, 25/02/2021, 26/02/2021, 27/02/2021, 28/02/2021, 01/03/2021, 02/03/2021, 03/03/2021, 04/03/2021, 05/03/2021, 06/03/2021, 07/03/2021, 08/03/2021, 09/03/2021, 10/03/2021, 11/03/2021, 12/03/2021, 13/03/2021, 14/03/2021, 15/03/2021, 16/03/2021, 17/03/2021, 18/03/2021, 19/03/2021, 20/03/2021, 21/03/2021, 22/03/2021, 23/03/2021, 24/03/2021, 25/03/2021, 26/03/2021, 27/03/2021, 28/03/2021, 29/03/2021, 30/03/2021, 31/03/2021, 01/04/2021, 02/04/2021, 03/04/2021, 04/04/2021, 05/04/2021, 06/04/2021, 07/04/2021, 08/04/2021, 09/04/2021, 10/04/2021, 11/04/2021, 12/04/2021, 13/04/2021, 14/04/2021, 15/04/2021, 16/04/2021, 17/04/2021, 18/04/2021, 19/04/2021, 20/04/2021, 21/04/2021, 22/04/2021, 23/04/2021, 24/04/2021, 25/04/2021, ...]",140,NA
State,object,0,"[India, Andaman and Nicobar Islands, Andhra Pradesh, Arunachal Pradesh, Assam, Bihar, Chandigarh, Chhattisgarh, Dadra and Nagar Haveli and Daman and Diu, Delhi, Goa, Gujarat, Haryana, Himachal Pradesh, Jammu and Kashmir, Jharkhand, Karnataka, Kerala, Ladakh, Lakshadweep, Madhya Pradesh, Maharashtra, Manipur, Meghalaya, Mizoram, Nagaland, Odisha, Puducherry, Punjab, Rajasthan, Sikkim, Tamil Nadu, Telangana, Tripura, Uttar Pradesh, Uttarakhand, West Bengal]",37,Chandigarh:140Maharashtra:139Himachal Pradesh:139Lakshadweep:139Tamil Nadu:139Goa:139Madhya Pradesh:139Manipur:139Tripura:139Jammu and Kashmir:139Karnataka:139Gujarat:139Uttarakhand:139Assam:139West Bengal:139Kerala:139Meghalaya:139Odisha:139Andaman and Nicobar Islands:139Punjab:139Sikkim:139Mizoram:139Arunachal Pradesh:139Bihar:139Chhattisgarh:139Telangana:139Uttar Pradesh:139Jharkhand:139Haryana:139Dadra and Nagar Haveli and Daman and Diu:139Rajasthan:139Nagaland:139India:139Puducherry:139Delhi:139Andhra Pradesh:139Ladakh:139
Total Individuals Vaccinated,float64,120,"[48276.0, 58604.0, 99449.0, 195525.0, 251280.0, 365965.0, 549381.0, 759008.0, 835058.0, 1277104.0, 1293784.0, 1726490.0, 2295491.0, 2814803.0, 3067736.0, 3127107.0, 3350265.0, 3527971.0, 3825835.0, 4314304.0, 4765924.0, 5111827.0, 5168099.0, 5615499.0, 6002474.0, 6462270.0, 6958553.0, 7423817.0, 7687352.0, 7710367.0, 8111162.0, 8337600.0, 8704698.0, 8862491.0, 9523567.0, 9734505.0, 9757217.0, 10069263.0, 10288293.0, 10517568.0, 10846192.0, 11141375.0, 11141426.0, 11141651.0, 11886320.0, 12553326.0, 13393538.0, 14415912.0, 15438495.0, 15541529.0, 16976138.0, 18360243.0, 19538547.0, 19953987.0, 21772159.0, 23159604.0, 23317750.0, 25618250.0, 25709257.0, 30011328.0, 31952935.0, 34228371.0, 36568988.0, 37303458.0, 39612614.0, 42148875.0, 44250191.0, 46454700.0, 48749313.0, 50865316.0, 51179587.0, 51774289.0, 53429356.0, 55188803.0, 57443466.0, 60936753.0, 63743700.0, 66838504.0, 69620234.0, 72035640.0, 76671812.0, 80998099.0, 84296847.0, 87411872.0, 90111528.0, 93559587.0, 95823614.0, 98703696.0, 100853671.0, 102927296.0, 104950409.0, 105896430.0, 108178574.0, 110165234.0, 111673361.0, 113584269.0, 115461202.0, 117105343.0, 117795008.0, 119886252.0, ...]",4786,NA
Total Sessions Conducted,float64,120,"[3455.0, 8532.0, 13611.0, 17855.0, 25472.0, 32226.0, 36988.0, 43076.0, 49851.0, 55151.0, 60821.0, 69495.0, 78523.0, 83664.0, 87822.0, 91593.0, 97432.0, 106461.0, 116568.0, 126714.0, 137110.0, 145121.0, 153231.0, 164116.0, 175556.0, 187224.0, 197650.0, 207633.0, 216559.0, 223423.0, 233527.0, 245131.0, 256706.0, 268275.0, 279055.0, 288334.0, 299726.0, 312382.0, 325527.0, 339617.0, 352882.0, 357440.0, 357450.0, 367595.0, 401961.0, 448977.0, 53325

In [28]:
tbl_report(coviddist)

Table Shape (800, 16)


,dtype,nulls,uniques,num_uniques,value_counts
SlNo,int64,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 101, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, ...]",765,NA
State_Code,object,0,"[UN, AN, AP, AR, AS, BR, CH, CT, DL, DN, GA, GJ, HP, HR, JH, JK, KA, KL, LA, LD, MH, ML, MN, MP, MZ, NL, OR, PB, PY, RJ, SK, TG, TN, TR, UP, UT, WB]",37,UP:76MP:54TN:41BR:39RJ:38MH:38AS:36TG:36GJ:35OR:32KA:32CT:30AR:26JH:25WB:25HR:25PB:23JK:23MN:18KL:16AP:16UT:14HP:13NL:13MZ:12DL:12ML:12TR:9PY:5SK:5DN:5GA:4AN:4LA:3CH:2LD:2UN:1
State,object,0,"[State Unassigned, Andaman and Nicobar Islands, Andhra Pradesh, Arunachal Pradesh, Assam, Bihar, Chandigarh, Chhattisgarh, Delhi, Dadra and Nagar Haveli and Daman and Diu, Goa, Gujarat, Himachal Pradesh, Haryana, Jharkhand, Jammu and Kashmir, Karnataka, Kerala, Ladakh, Lakshadweep, Maharashtra, Meghalaya, Manipur, Madhya Pradesh, Mizoram, Nagaland, Odisha, Punjab, Puducherry, Rajasthan, Sikkim, Telangana, Tamil Nadu, Tripura, Uttar Pradesh, Uttarakhand, West Bengal]",37,Uttar Pradesh:76Madhya Pradesh:54Tamil Nadu:41Bihar:39Rajasthan:38Maharashtra:38Telangana:36Assam:36Gujarat:35Karnataka:32Odisha:32Chhattisgarh:30Arunachal Pradesh:26Jharkhand:25West Bengal:25Haryana:25Jammu and Kashmir:23Punjab:23Manipur:18Andhra Pradesh:16Kerala:16Uttarakhand:14Nagaland:13Himachal Pradesh:13Delhi:12Mizoram:12Meghalaya:12Tripura:9Sikkim:5Puducherry:5Dadra and Nagar Haveli and Daman and Diu:5Andaman and Nicobar Islands:4Goa:4Ladakh:3Chandigarh:2Lakshadweep:2State Unassigned:1
District_Key,object,0,"[UN_Unassigned, AN_Nicobars, AN_North and Middle Andaman, AN_South Andaman, AP_Foreign Evacuees, AP_Anantapur, AP_Chittoor, AP_East Godavari, AP_Guntur, AP_Krishna, AP_Kurnool, AP_Other State, AP_Prakasam, AP_S.P.S. Nellore, AP_Srikakulam, AP_Visakhapatnam, AP_Vizianagaram, AP_West Godavari, AP_Y.S.R. Kadapa, AR_Anjaw, AR_Changlang, AR_East Kameng, AR_East Siang, AR_Kamle, AR_Kra Daadi, AR_Kurung Kumey, AR_Lepa Rada, AR_Lohit, AR_Longding, AR_Lower Dibang Valley, AR_Lower Siang, AR_Lower Subansiri, AR_Namsai, AR_Pakke Kessang, AR_Papum Pare, AR_Shi Yomi, AR_Siang, AR_Tawang, AR_Tirap, AR_Upper Dibang Valley, AR_Upper Siang, AR_Upper Subansiri, AR_West Kameng, AR_West Siang, AS_Airport Quarantine, AS_Baksa, AS_Barpeta, AS_Biswanath, AS_Bongaigaon, AS_Cachar, AS_Charaideo, AS_Chirang, AS_Darrang, AS_Dhemaji, AS_Dhubri, AS_Dibrugarh, AS_Dima Hasao, AS_Goalpara, AS_Golaghat, AS_Hailakandi, AS_Hojai, AS_Jorhat, AS_Kamrup, AS_Kamrup Metropolitan, AS_Karbi Anglong, AS_Karimganj, AS_Kokrajhar, AS_Lakhimpur, AS_Majuli, AS_Morigaon, AS_Nagaon, AS_Nalbari, AS_Other State, AS_Sivasagar, AS_Sonitpur, AS_South Salmara Mankachar, AS_Tinsukia, AS_Udalguri, AS_West Karbi Anglong, BR_Araria, BR_Arwal, BR_Aurangabad, BR_Banka, BR_Begusarai, BR_Bhagalpur, BR_Bhojpur, BR_Buxar, BR_Darbhanga, BR_East Champaran, BR_Gaya, BR_Gopalganj, BR_Jamui, BR_Jehanabad, BR_Kaimur, BR_Katihar, BR_Khagaria, BR_Kishanganj, BR_Lakhisarai, BR_Madhepura, BR_Madhubani, ...]",800,NA
District,object,0,"[Unassigned, Nicobars, North and Middle Andaman, South Andaman, Foreign Evacuees, Anantapur, Chittoor, East Godavari, Guntur, Krishna, Kurnool, Other State, Prakasam, S.P.S. Nellore, Srikakulam, Visakhapatnam, Vizianagaram, West Godavari, Y.S.R. Kadapa, Anjaw, Changlang, East Kameng, East Siang, Kamle, Kra Daadi, Kurung Kumey, Lepa Rada, Lohit, Longding, Lower Dibang Valley, Lower Siang, Lower Subansiri, Namsai, Pakke Kessang, Papum Pare, Shi Yomi, Siang, Tawang, Tirap, Upper Dibang Valley, Upper Siang, Upper Subansiri, West Kameng, West Siang, Airport Quarantine, Baksa, Barpeta, Biswanath, Bongaigaon, Cachar, Charaideo, Chirang, Darrang, Dhemaji, Dhubri, Dibrugarh, Dima Hasao, Go

In [30]:
tbl_report(coviddaily)

Table Shape (5694, 7)


,dtype,nulls,uniques,num_uniques,value_counts
Unnamed: 0,int64,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]",5694,NA
Date,object,0,"[01-Apr-20, 01-Aug-20, 01-Jul-20, 01-Jun-20, 01-May-20, 02-Apr-20, 02-Aug-20, 02-Jul-20, 02-Jun-20, 02-May-20, 03-Apr-20, 03-Aug-20, 03-Jul-20, 03-Jun-20, 03-May-20, 04-Apr-20, 04-Aug-20, 04-Jul-20, 04-Jun-20, 04-May-20, 05-Apr-20, 05-Aug-20, 05-Jul-20, 05-Jun-20, 05-May-20, 06-Apr-20, 06-Aug-20, 06-Jul-20, 06-Jun-20, 06-May-20, 07-Apr-20, 07-Jul-20, 07-Jun-20, 07-May-20, 08-Apr-20, 08-Jul-20, 08-Jun-20, 08-May-20, 09-Apr-20, 09-Jul-20, 09-Jun-20, 09-May-20, 10-Apr-20, 10-Jul-20, 10-Jun-20, 10-May-20, 11-Apr-20, 11-Jul-20, 11-Jun-20, 11-May-20, 12-Apr-20, 12-Jul-20, 12-Jun-20, 12-May-20, 13-Apr-20, 13-Jul-20, 13-Jun-20, 13-May-20, 14-Apr-20, 14-Jul-20, 14-Jun-20, 14-Mar-20, 14-May-20, 15-Apr-20, 15-Jul-20, 15-Jun-20, 15-Mar-20, 15-May-20, 16-Apr-20, 16-Jul-20, 16-Jun-20, 16-Mar-20, 16-May-20, 17-Apr-20, 17-Jul-20, 17-Jun-20, 17-Mar-20, 17-May-20, 18-Apr-20, 18-Jul-20, 18-Jun-20, 18-Mar-20, 18-May-20, 19-Apr-20, 19-Jul-20, 19-Jun-20, 19-Mar-20, 19-May-20, 20-Apr-20, 20-Jul-20, 20-Jun-20, 20-Mar-20, 20-May-20, 21-Apr-20, 21-Jul-20, 21-Jun-20, 21-Mar-20, 21-May-20, 22-Apr-20, 22-Jul-20, ...]",146,NA
State,object,0,"[AN, AP, AR, AS, BR, CH, CT, DD, DL, DN, GA, GJ, HP, HR, JH, JK, KA, KL, LA, LD, MH, ML, MN, MP, MZ, NL, OR, PB, PY, RJ, SK, TG, TN, TR, TT, UN, UP, UT, WB]",39,DD:146AN:146GA:146AR:146BR:146TR:146CH:146OR:146KA:146MZ:146PB:146AP:146MN:146MH:146TG:146RJ:146UN:146KL:146DL:146TN:146WB:146TT:146HR:146GJ:146NL:146JK:146UT:146ML:146JH:146DN:146AS:146HP:146LA:146CT:146UP:146MP:146LD:146PY:146SK:146
Confirmed,int64,0,"[0, 67, 15, 3, 2, 32, 13, 7, 9, 24, 33, 1, 4, 27, 30, 110, 424, 88, 9276, 83, 1457, 3521, 28, 235, 1118, 45, 280, 1136, 70, 793, 790, 613, 5172, 1129, 58, 9601, 135, 808, 5, 138, 1602, 944, 134, 1160, 11, 2083, 5879, 252, 55117, 3587, 264, 2589, 657, 548, 216, 6, 82, 2442, 16, 72, 675, 26, 393, 35, 198, 1272, 151, 17, 5537, 268, 42, 251, 100, 25, 304, 1018, 3882, 8, 19429, -89, 564, 66, 611, 105, 18, 146, 49, 990, 423, 265, 155, 187, 57, 2358, 12, 194, 156, 38, 269, 94, ...]",1268,NA
Deceased,int64,0,"[0, 3, 1, 9, 2, 58, 14, 26, 24, 7, 8, 11, 98, 322, 19, 99, 854, 47, 48, 6, 5, 61, 21, 4, 198, 63, 438, 15, 50, 25, 76, -1, 201, 22, 77, 67, 10, 84, 260, 18, 12, 760, 53, 49, 125, 57, 378, 17, 16, 33, 29, 103, 13, 222, 36, 92, 266, 109, 806, 59, 64, 444, 30, 122, 259, 28, 27, 140, 74, 110, 300, 20, 108, 849, 39, 54, 81, 42, 295, 65, 611, 44, 123, 274, 35, 23, 100, 334, -3, 804, 40, 37, 151, 60, 421, 139, 286, 34, 128, 72, ...]",241,NA
Recovered,int64,0,"[0, 1, 10, 2, 16, 12, 12750, 51, 1085, 1823, 380, 1201, 30, 227, 875, 43, 853, 170, 654, 3860, 752, 13, 10725, 37, 698, 6, 5, 756, 341, 98, 823, 38, 1114, 7010, 136, 51368, 2471, 162, 2143, 477, 4, 184, 267, 3, 53, 1644, 27, 74, 368, 42, 527, 47, 134, 145, 132, 46, 2243, 26, 260, 14, 164, 122, 29, 354, 788, 2852, 12064, 545, 86, 398, 34, 99, 221, 15, 7, 268, 861, 40, 19, 100, 18, 779, 161, 119, 181, 63, 413, 3882, 187, 120, 149, 82, 73, 123, 31, 22, 9, 106, 223, 54, ...]",1033,NA
State_Name,object,0,"[Andaman and Nicobar Islands, Andhra Pradesh, Arunachal Pradesh, Assam, Bihar, Chandigarh, Chhattisgarh, Daman and Diu, Delhi, Dadra and Nagar Haveli and Daman and Diu, Goa, Gujarat, Himachal Pradesh, Haryana, Jharkhand, Jammu and Kashmir, Karnataka, Kerala, Ladakh, Lakshadweep, Maharashtra, Meghalaya, Manipur, Madhya Pradesh, Mizoram, Nagaland, Odisha, Punjab, Puducherry, Rajasthan, Sikkim, Telangana, Tamil Nadu, Tripura, Total, State Unassigned, Uttar Pradesh, Uttarakhand, West Bengal]",39,Ma

In [31]:
tbl_report(covidtests)

Table Shape (3938, 28)


,dtype,nulls,uniques,num_uniques,value_counts
Updated On,object,0,"[17/04/2020, 24/04/2020, 27/04/2020, 01/05/2020, 16/05/2020, 19/05/2020, 20/05/2020, 21/05/2020, 22/05/2020, 23/05/2020, 24/05/2020, 25/05/2020, 26/05/2020, 27/05/2020, 28/05/2020, 29/05/2020, 30/05/2020, 31/05/2020, 01/06/2020, 02/06/2020, 03/06/2020, 04/06/2020, 05/06/2020, 06/06/2020, 07/06/2020, 08/06/2020, 09/06/2020, 10/06/2020, 11/06/2020, 12/06/2020, 13/06/2020, 14/06/2020, 15/06/2020, 16/06/2020, 17/06/2020, 18/06/2020, 19/06/2020, 20/06/2020, 21/06/2020, 22/06/2020, 23/06/2020, 24/06/2020, 25/06/2020, 26/06/2020, 27/06/2020, 28/06/2020, 29/06/2020, 30/06/2020, 01/07/2020, 02/07/2020, 03/07/2020, 04/07/2020, 05/07/2020, 06/07/2020, 07/07/2020, 08/07/2020, 09/07/2020, 10/07/2020, 11/07/2020, 12/07/2020, 13/07/2020, 14/07/2020, 15/07/2020, 16/07/2020, 17/07/2020, 18/07/2020, 19/07/2020, 20/07/2020, 21/07/2020, 22/07/2020, 23/07/2020, 24/07/2020, 25/07/2020, 26/07/2020, 27/07/2020, 28/07/2020, 29/07/2020, 30/07/2020, 31/07/2020, 01/08/2020, 02/08/2020, 03/08/2020, 04/08/2020, 05/08/2020, 06/08/2020, 02/04/2020, 10/04/2020, 11/04/2020, 12/04/2020, 13/04/2020, 14/04/2020, 15/04/2020, 16/04/2020, 18/04/2020, 19/04/2020, 20/04/2020, 21/04/2020, 22/04/2020, 23/04/2020, 25/04/2020, ...]",129,NA
State,object,0,"[Andaman and Nicobar Islands, Andhra Pradesh, Arunachal Pradesh, Assam, Bihar, Chandigarh, Chhattisgarh, Dadra and Nagar Haveli and Daman and Diu, Delhi, Goa, Gujarat, Haryana, Himachal Pradesh, Jammu and Kashmir, Jharkhand, Karnataka, Kerala, Ladakh, Madhya Pradesh, Maharashtra, Manipur, Meghalaya, Mizoram, Nagaland, Odisha, Puducherry, Punjab, Rajasthan, Sikkim, Tamil Nadu, Telangana, Tripura, Uttar Pradesh, Uttarakhand, West Bengal]",35,Kerala:129West Bengal:125Odisha:124Rajasthan:124Madhya Pradesh:124Haryana:124Maharashtra:123Uttarakhand:123Delhi:123Tamil Nadu:123Punjab:123Karnataka:123Bihar:123Uttar Pradesh:122Andhra Pradesh:121Nagaland:121Gujarat:121Jammu and Kashmir:121Chandigarh:120Himachal Pradesh:120Mizoram:118Jharkhand:118Goa:116Chhattisgarh:115Puducherry:112Arunachal Pradesh:111Assam:106Tripura:103Meghalaya:99Dadra and Nagar Haveli and Daman and Diu:97Sikkim:92Andaman and Nicobar Islands:85Ladakh:82Manipur:72Telangana:55
Total Tested,float64,61,"[1403.0, 2679.0, 2848.0, 3754.0, 6677.0, 6965.0, 7082.0, 7167.0, 7263.0, 7327.0, 7363.0, 7448.0, 7499.0, 7519.0, 7567.0, 7706.0, 7805.0, 8086.0, 8295.0, 8413.0, 8694.0, 9037.0, 9242.0, 9341.0, 9859.0, 10010.0, 10226.0, 10697.0, 10955.0, 11356.0, 11518.0, 11809.0, 12239.0, 12622.0, 12930.0, 13320.0, 13434.0, 13511.0, 13723.0, 13994.0, 14277.0, 14583.0, 14851.0, 15094.0, 15410.0, 15709.0, 15982.0, 16278.0, 16626.0, 16832.0, 16909.0, 17095.0, 17381.0, 17643.0, 17852.0, 18046.0, 18315.0, 18447.0, 18609.0, 18884.0, 19061.0, 19227.0, 19421.0, 19671.0, 19904.0, 20301.0, 20642.0, 21043.0, 21310.0, 21743.0, 21965.0, 22049.0, 22548.0, 22925.0, 23217.0, 23615.0, 24038.0, 24308.0, 24580.0, 25034.0, 25492.0, 25826.0, nan, 1800.0, 6374.0, 6958.0, 8755.0, 10505.0, 11613.0, 20235.0, 21450.0, 26958.0, 30733.0, 35755.0, 41512.0, 48032.0, 54338.0, 61266.0, 68034.0, 74551.0, ...]",3765,NA
Tag (Total Tested),object,29,"[Samples Sent, nan, Tested, Samples Collected, Reports Received]",5,Tested:1963Samples Sent:1159Reports Received:430Samples Collected:357nan:29
Positive,float64,75,"[12.0, 27.0, 33.0, 35.0, 38.0, 41.0, 44.0, 45.0, 47.0, 48.0, 50.0, 56.0, 59.0, 72.0, 73.0, 83.0, 90.0, 97.0, 100.0, 109.0, 116.0, 119.0, 125.0, 141.0, 147.0, 149.0, 151.0, 156.0, 163.0, 166.0, 171.0, 176.0, 180.0, 194.0, 198.0, 203.0, 207.0, 212.0, 221.0, 240.0, 259.0, 290.0, 324.0, 338.0, 363.0, 428.0, 471.0, 548.0, 636.0, 734.0, 830.0, 928.0, 1027.0, nan, 132.0, 365.0, 381.0, 405.0, 432.0, 473.0, 525.0, 534.0, 572.0, 603.0, 647.0, 722.0, 757.0, 813.0, 893.0, 955.0, 1016.0, 1097.0, 1177.0, 1259.0, 1332.0, 1403.0, 1463.0, 1525.0, 1583.0, 1650.0, 1717.0, 1777.0, 1833.0, 1887.0, 1930.0, 1980.0, 2018.0, 2051.0, 2137.0, 2100.0, 2157.0, 2230.0, 2282.0

In [ ]:
#clearing all nans and imputations are being carried out
aqi.dropna(axis=0, how='all', thresh=None, subset=None, inplace=True)
numerical_features = [c for c, dtype in zip(aqi.columns, aqi.dtypes)
                 if dtype.kind in ['i','f']]
categorical_features = [c for c, dtype in zip(aqi.columns, aqi.dtypes)
                 if dtype.kind not in ['i','f']]
kimput=KNNImputer(n_neighbors=2, weights="uniform")
simput=SimpleImputer(strategy = 'most_frequent', fill_value = 'missing')
aqi[numerical_features]=kimput.fit_transform(df[numerical_features])
aqi[categorical_features]=simput.fit_transform(df[categorical_features])
    
    

In [ ]:
#datasets for air quality index of different years
aqi15=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2015]
aqi16=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2016]
aqi17=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2017]
aqi18=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2018]
aqi19=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2019]
aqi20=aqi[pd.DatetimeIndex(aqi['Datetime']).year==2020]
#creating lists to divide dataframes as and according to requirements
df_list=[aqi15,aqi16,aqi17,aqi18,aqi19]
df_list2=[aqi20,covidstate,coviddist,coviddaily,covidtests]

In [ ]:
tbl_report(aqi15)

In [ ]:
tbl_report(aqi16)

In [ ]:
tbl_report(aqi17)

In [ ]:
tbl_report(aqi18)

In [ ]:
tbl_report(aqi19)